In [1]:
import os
import sys
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Define and set the working directory
new_directory = '/content/drive/Shareddrives/OM in Business Analytics/OM in BA - Project/Codice/03 - Indici'
if os.path.isdir(new_directory):
    os.chdir(new_directory)
    sys.path.append(new_directory)
    print("Current directory:", os.getcwd())
else:
    print("Directory does not exist:", new_directory)

Mounted at /content/drive
Current directory: /content/drive/Shareddrives/OM in Business Analytics/OM in BA - Project/Codice/03 - Indici


# Load data

In [2]:
import geopandas as gpd

# Load the base GeoDataFrame containing spatial and attribute data
gdf = gpd.read_file("/content/drive/Shareddrives/OM in Business Analytics/OM in BA - Project/Codice/03 - Indici/gdf_finale_ind.gpkg")

# Load the GeoDataFrame for the conservative 2030 scenario
gdf_conservativo = gpd.read_file("/content/drive/Shareddrives/OM in Business Analytics/OM in BA - Project/Codice/03 - Indici/gdf_2030_conservativo_finale_ind.gpkg")

# Load the GeoDataFrame for the optimistic 2050 scenario
gdf_ottimistico = gpd.read_file("/content/drive/Shareddrives/OM in Business Analytics/OM in BA - Project/Codice/03 - Indici/gdf_2030_ottimistico_finale_ind.gpkg")

# ACTUAL MODELS

## FIRST MODEL
## Charging Station Allocation Model – Hexagon-Based

### Objective  
Allocate *n* charging stations among the hexagons in the province of Brescia to **maximize the coverage of daily electricity demand** (in kWh), considering that each station has a fixed delivery capacity.

---

### Decision Variables  
- $ x_h \in \mathbb{Z}_{\geq 0} $: Number of charging stations installed in hexagon h
- $ y_h \in [0, D_h] $: Amount of daily demand covered in hexagon h

---

### Parameters  
- $ D_h $: Daily electricity demand in hexagon h   
- $ \text{cap} = 555.43 $: Capacity of a single charging station (kWh/day)  
- $ p $: Maximum number of charging stations to be installed  
- $ n$: Total number of hexagons

---

### Constraints  

1. **Total station limit**  
   $
   \sum_{h=1}^{n} x_h \leq p
  $

2. **Demand coverage cannot exceed actual demand**  
   $
   y_h \leq D_h \quad \forall h
   $

3. **Demand coverage limited by installed station capacity**  
  $
   y_h \leq \text{cap} \cdot x_h \quad \forall h
   $

---

### Objective Function  

Maximize total covered demand:  
$
\max \sum_{h=1}^{n} y_h
$

This ensures that charging stations are placed to maximize the total amount of demand served across all hexagons.


In [ ]:
pip install pulp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 97.1 MB/s eta 0:00:00


### Model function

In [ ]:
import pulp

def ottimizza_colonnine_esagoni(gdf_mappa, p_colonnine=30, cap=555.43, verbose=True):
    """
    Solves the optimal allocation problem of charging stations for H3 hexagons.

    Parameters:
    - gdf_mappa: GeoDataFrame with at least 'h3_id' and 'Weighted Daily Demand (kWh)'
    - p_colonnine: total number of available charging stations
    - cap: daily capacity of a single charging station (kWh)
    - verbose: if True, prints the results

    Returns:
    - GeoDataFrame with additional columns: 'Allocated Stations', 'Covered Demand (kWh)'
    """
    df = gdf_mappa.copy()
    demand = df["Domanda Giornaliera Ponderata (kWh)"].tolist()
    n = len(demand)

    # Model
    model = pulp.LpProblem("Colonnine_multiple_per_esagono", pulp.LpMaximize)

    # Decision variables
    x = pulp.LpVariable.dicts("Colonnine", range(n), lowBound=0, cat="Integer")
    y = pulp.LpVariable.dicts("DomandaCoperta", range(n), lowBound=0)

    # Objective
    model += pulp.lpSum(y[i] for i in range(n))

    # Constraint: maximum total number of charging stations
    model += pulp.lpSum(x[i] for i in range(n)) <= p_colonnine

    # Constraints for each hexagon
    for i in range(n):
        model += y[i] <= demand[i]
        model += y[i] <= cap * x[i]

    # Solve
    model.solve()

    # Extract results
    colonnine_allocate = [int(x[i].varValue) if x[i].varValue else 0 for i in range(n)]
    domanda_coperta = [y[i].varValue if y[i].varValue else 0.0 for i in range(n)]

    df["Colonnine allocate"] = colonnine_allocate
    df["Domanda coperta (kWh)"] = domanda_coperta

    if verbose:
        for i in range(n):
            if colonnine_allocate[i] > 0:
                print(f"H3 {df.iloc[i]['h3_id']}: {colonnine_allocate[i]} colonnine, "
                      f"{domanda_coperta[i]:.1f} kWh coperti su {demand[i]:.1f} kWh")
        print(f"\nDomanda totale coperta: {pulp.value(model.objective):.2f} kWh")

    return df

### Map function

In [ ]:
import folium
import numpy as np
import branca.colormap as cm

def crea_mappa_colonnine_esagoni(gdf,
                                 colonna_domanda="Domanda Giornaliera Ponderata (kWh)",
                                 colonna_coperta="Domanda coperta (kWh)",
                                 colonna_colonnine="Colonnine allocate",
                                 nome_mappa="Mappa Colonnine",
                                 location=[45.54, 10.22],
                                 zoom_start=10):
    """
    Creates a Folium map of H3 hexagons with colors based on the NUMBER OF ALLOCATED CHARGING STATIONS.
    Tooltips show information on covered demand and allocated stations.

    Parameters:
    - gdf: GeoDataFrame with geometries and required columns
    - colonna_domanda: name of the column with daily demand
    - colonna_coperta: name of the column with covered demand (model output)
    - colonna_colonnine: name of the column with allocated charging stations
    - nome_mappa: title of the map
    - location: initial coordinates for the map center
    - zoom_start: initial zoom level

    Returns:
    - `folium.Map` object
    """

    # Prepare the logarithmic scale based on number of allocated charging stations
    colonnine_vals = gdf[colonna_colonnine].clip(lower=1e-2)  # Avoid log(0)
    log_min = np.log10(colonnine_vals.min())
    log_max = np.log10(colonnine_vals.max())
    colormap = cm.linear.YlGnBu_09.scale(log_min, log_max).to_step(n=10)

    # Create the map
    mappa = folium.Map(location=location, zoom_start=zoom_start, tiles="cartodbpositron")

    # Add polygons
    for _, row in gdf.iterrows():
        domanda = row.get(colonna_domanda, 0)
        coperta = row.get(colonna_coperta, 0)
        colonnine = row.get(colonna_colonnine, 0)

        # Color based on log of allocated stations
        color = colormap(np.log10(max(colonnine, 1e-2)))

        tooltip_text = (
            f"<b>H3 ID: {row.get('h3_id', 'N/A')}</b><br>"
            f"Domanda: {domanda:.1f} kWh<br>"
            f"Coperta: {coperta:.1f} kWh<br>"
            f"Colonnine allocate: {int(colonnine)}"
        )

        folium.GeoJson(
            row["geometry"],
            style_function=lambda feature, color=color: {
                "fillColor": color,
                "color": "black",
                "weight": 0.5,
                "fillOpacity": 0.7,
            },
            tooltip=folium.Tooltip(tooltip_text)
        ).add_to(mappa)

    # Add legend
    colormap.caption = f"{colonna_colonnine} (logarithmic scale)"
    colormap.add_to(mappa)

    return mappa

In [ ]:
# Run the optimization on H3 cells
gdf_risultati_base = ottimizza_colonnine_esagoni(gdf, p_colonnine=1623)

H3 881f99274bfffff: 1 colonnine, 29.5 kWh coperti su 29.5 kWh
H3 881f992215fffff: 1 colonnine, 37.4 kWh coperti su 37.4 kWh
H3 881f992213fffff: 1 colonnine, 67.3 kWh coperti su 67.3 kWh
H3 881f9935c7fffff: 1 colonnine, 18.6 kWh coperti su 18.6 kWh
H3 881f9935a1fffff: 1 colonnine, 17.8 kWh coperti su 17.8 kWh
H3 881f9922ddfffff: 1 colonnine, 106.8 kWh coperti su 106.8 kWh
H3 881f992285fffff: 1 colonnine, 50.0 kWh coperti su 50.0 kWh
H3 881f9922e3fffff: 1 colonnine, 18.2 kWh coperti su 18.2 kWh
H3 881f992217fffff: 1 colonnine, 20.1 kWh coperti su 20.1 kWh
H3 881f993501fffff: 1 colonnine, 71.3 kWh coperti su 71.3 kWh
H3 881f99353bfffff: 1 colonnine, 108.0 kWh coperti su 108.0 kWh
H3 881f992295fffff: 1 colonnine, 51.4 kWh coperti su 51.4 kWh
H3 881f9922cdfffff: 1 colonnine, 76.6 kWh coperti su 76.6 kWh
H3 881f9920d9fffff: 1 colonnine, 44.7 kWh coperti su 44.7 kWh
H3 881f992745fffff: 1 colonnine, 46.8 kWh coperti su 46.8 kWh
H3 881f992291fffff: 1 colonnine, 70.6 kWh coperti su 70.6 kWh
H3 8

### Map visualisation

In [ ]:
# Create and display the map based on hexagons
mappa = crea_mappa_colonnine_esagoni(gdf_risultati_base)
mappa

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# Save the map as an HTML file
mappa.save("/content/drive/Shareddrives/OM in Business Analytics/OM in BA - Project/Codice/04 - Modello/log_heatmap_first_model_def.html")

### Results

In [ ]:
total_allocated_chargers = gdf_risultati_base["Colonnine allocate"].sum()
total_covered_demand = gdf_risultati_base["Domanda coperta (kWh)"].sum()
total_demand = gdf_risultati_base["Domanda Giornaliera Ponderata (kWh)"].sum()

print(f"Total allocated charging stations: {total_allocated_chargers}")
print(f"Total covered demand (kWh): {total_covered_demand:.2f}")
print(f"Total demand (kWh): {total_demand:.2f}")

Total allocated charging stations: 1623
Total covered demand (kWh): 28010.36
Total demand (kWh): 34500.45


## SECOND MODEL  
## Charging Station Allocation Model – Aggregated Neighborhood Demand (Hexagon-Based)

### Objective  
Allocate *n* charging stations among the hexagons in the province of Brescia to **maximize the coverage of local daily electricity demand**, allowing each hexagon to benefit from stations located in **neighboring hexagons** as well.

---

### Decision Variables  
- $ x_h \in \mathbb{Z}_{\geq 0} $: Number of charging stations installed in hexagon *h*  
- $ y_h \in [0, D_h] $: Amount of daily demand covered in hexagon *h*

---

### Parameters  
- $ D_h $: Daily electricity demand in hexagon *h*  
- $ \text{cap} = 555.43 $: Daily capacity of a single charging station (kWh/day)  
- $ p $: Maximum number of charging stations to be installed  
- $ \mathcal{N}(h) $: Set of neighbors of hexagon *h* (within k-ring 1)

---

### Constraints  

1. **Total station limit**  
   $$
   \sum_{h=1}^{n} x_h \leq p
   $$

2. **Demand coverage cannot exceed actual local demand**  
   $$
   y_h \leq D_h \quad \forall h
   $$

3. **Coverage limited by capacity of stations in *h* and its neighbors**  
   $$
   y_h \leq \text{cap} \cdot \left( x_h + \sum_{j \in \mathcal{N}(h)} x_j \right) \quad \forall h
   $$

---

### Objective Function  

Maximize total local demand coverage:  
$$
\max \sum_{h=1}^{n} y_h
$$

This model prioritizes local demand fulfillment while allowing neighboring station capacity to contribute, thereby improving spatial equity in charging infrastructure deployment.

In [6]:
!pip install h3==3.*

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 13.0 MB/s eta 0:00:00


### Neighbor list

In [7]:
import h3

# Get the unique list of hexagons in your GeoDataFrame (gdf)
h3_ids_present = set(gdf['h3_id'])

# Create a dictionary {index in gdf : list of neighboring indices}
h3_id_to_idx = {h3_id: idx for idx, h3_id in enumerate(gdf['h3_id'])}

neighbors_dict = {}

for idx, row in gdf.iterrows():
    h3_id = row['h3_id']
    # Get the neighboring H3 hexagons (k_ring with radius 1)
    neighbors_h3 = h3.k_ring(h3_id, 1)
    # Filter only those present in your dataframe
    present_neighbors = [h3_id_to_idx[v] for v in neighbors_h3 if v in h3_ids_present and v != h3_id]
    neighbors_dict[idx] = present_neighbors

print(f"Neighbor dictionary created with {len(neighbors_dict)} hexagons")

Neighbor dictionary created with 6495 hexagons


### Model function

In [3]:
pip install pulp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 102.5 MB/s eta 0:00:00


In [8]:
import pulp
import pandas as pd

def ottimizza_colonnine_con_domanda_aggregata(gdf_mappa, neighbours_dict= neighbors_dict, p_colonnine=30, cap=555.43, verbose=True ):
    """
    Risolve il problema di allocazione ottimale delle colonnine considerando la domanda aggregata nel vicinato.

    Parametri:
    - gdf_mappa: GeoDataFrame con 'h3_id' e 'Domanda Giornaliera Ponderata (kWh)'
    - neighbours_dict: dizionario {h3_id: lista di vicini h3_id}
    - p_colonnine: numero totale di colonnine disponibili
    - cap: capacità giornaliera di una colonnina (kWh)
    - verbose: se True, stampa i risultati

    Ritorna:
    - GeoDataFrame con colonne aggiuntive: 'Colonnine allocate', 'Domanda coperta (kWh)', 'Domanda aggregata (kWh)'
    """

    df = gdf_mappa.copy()
    h3_list = df['h3_id'].tolist()
    h3_to_idx = {h3_id: idx for idx, h3_id in enumerate(h3_list)}

    demand = df["Domanda Giornaliera Ponderata (kWh)"].tolist()
    n = len(demand)

    # Calculate aggregated demand (hexagon + neighbors)
    domanda_aggregata = []
    for h3_id in h3_list:
        idx = h3_to_idx[h3_id]
        vicini = neighbours_dict.get(h3_id, [])
        domanda_sum = demand[idx]  # local demand
        for vicino_id in vicini:
            if vicino_id in h3_to_idx:  # only consider if neighbor is in the dataset
                domanda_sum += demand[h3_to_idx[vicino_id]]
        domanda_aggregata.append(domanda_sum)
    df["Domanda aggregata (kWh)"] = domanda_aggregata

    # Model
    model = pulp.LpProblem("Colonnine_con_domanda_aggregata", pulp.LpMaximize)

    # Decision variables
    x = pulp.LpVariable.dicts("Colonnine", range(n), lowBound=0, cat="Integer")  # allocated stations
    y = pulp.LpVariable.dicts("DomandaCoperta", range(n), lowBound=0)            # covered demand

    # Objective: maximize total covered demand
    model += pulp.lpSum(y[i] for i in range(n))

    # Constraint: maximum total number of stations
    model += pulp.lpSum(x[i] for i in range(n)) <= p_colonnine

    # Constraints for each hexagon
    for i in range(n):
        h3_id = h3_list[i]
        vicini = neighbours_dict.get(h3_id, [])
        vicini_idx = [h3_to_idx[v] for v in vicini if v in h3_to_idx]

        # Vincolo: massimo 10 colonnine per esagono
        model += x[i] <= 10

        # Covered demand must be <= local demand
        model += y[i] <= demand[i]

        # Covered demand must be <= total capacity of stations allocated on hexagon + neighbors
        model += y[i] <= cap * (x[i] + pulp.lpSum(x[j] for j in vicini_idx))

    # Solve
    model.solve()

    # Extract results
    colonnine_allocate = [int(x[i].varValue) if x[i].varValue else 0 for i in range(n)]
    domanda_coperta = [y[i].varValue if y[i].varValue else 0.0 for i in range(n)]

    df["Colonnine allocate"] = colonnine_allocate
    df["Domanda coperta (kWh)"] = domanda_coperta

    if verbose:
        for i in range(n):
            if colonnine_allocate[i] > 0:
                print(f"H3 {df.iloc[i]['h3_id']}: {colonnine_allocate[i]} colonnine, "
                      f"{domanda_coperta[i]:.1f} kWh coperti su {demand[i]:.1f} kWh (Domanda agg: {domanda_aggregata[i]:.1f} kWh)")
        print(f"\nDomanda totale coperta: {pulp.value(model.objective):.2f} kWh")

    return df

### Map function

In [12]:
import folium

def crea_mappa_colonnine_discrete(gdf,
                                  colonna_domanda="Domanda Giornaliera Ponderata (kWh)",
                                  colonna_coperta="Domanda coperta (kWh)",
                                  colonna_colonnine="Colonnine allocate",
                                  nome_mappa="Mappa Colonnine",
                                  location=[45.54, 10.22],
                                  zoom_start=10):
    """
    Creates a Folium map of H3 hexagons with discrete colors based on the NUMBER OF CHARGING STATIONS allocated:
    0 -> gray, 1 -> light blue, 2 -> bright blue, 3 -> light navy blue, 4 -> dark blue, 5+ -> purple.
    """

    # Check if the column exists
    if colonna_colonnine not in gdf.columns:
        raise ValueError(f"Column '{colonna_colonnine}' not found in the GeoDataFrame.")

    # Discrete color map
    def get_color(colonnine):
        if colonnine == 0:
            return "#B0B0B0"  # gray
        elif colonnine == 1:
            return "#ADD8E6"  # light blue
        elif colonnine == 2:
            return "#0096FF"  # bright blue
        elif colonnine == 3:
            return "#0000FF"  # light navy blue
        elif colonnine == 4:
            return "#00008B"  # dark blue
        else:
            return "#800080"  # purple (5 or more)

    # Create the map
    mappa = folium.Map(location=location, zoom_start=zoom_start, tiles="cartodbpositron")

    # Add polygons
    for _, row in gdf.iterrows():
        # Get the number of charging stations, use default if missing
        colonnine = int(row.get(colonna_colonnine, 0))

        # Do not modify the demand if there are 0 charging stations (avoid zeroing it)
        domanda = row.get(colonna_domanda, 0)
        if colonnine == 0 and domanda == 0:
            domanda = row.get(colonna_domanda, 0)  # Prevent zeroing if no stations

        coperta = row.get(colonna_coperta, 0)

        # Compute color based on the number of charging stations
        color = get_color(colonnine)

        # Create tooltip with details
        tooltip_text = (
            f"<b>H3 ID: {row.get('h3_id', 'N/A')}</b><br>"
            f"Demand: {domanda:.1f} kWh<br>"
            f"Covered: {coperta:.1f} kWh<br>"
            f"Allocated charging stations: {colonnine}"
        )

        # Add polygon to the map
        folium.GeoJson(
            row["geometry"],
            style_function=lambda feature, color=color: {
                "fillColor": color,
                "color": "black",
                "weight": 0.5,
                "fillOpacity": 0.7,
            },
            tooltip=folium.Tooltip(tooltip_text)
        ).add_to(mappa)

    # HTML per la legenda basata sui colori attuali
    legenda_html = """
     <div style="
         position: fixed;
         bottom: 50px; left: 50px; width: 220px; height: auto;
         background-color: white;
         border:2px solid grey;
         z-index:9999;
         font-size:14px;
         padding: 10px;
         box-shadow: 2px 2px 6px rgba(0,0,0,0.3);
     ">
     <b>Legenda - Colonnine allocate</b><br>
     <i style="background:#B0B0B0;width:15px;height:15px;float:left;margin-right:5px;display:inline-block;"></i> 0 (nessuna)<br>
     <i style="background:#ADD8E6;width:15px;height:15px;float:left;margin-right:5px;display:inline-block;"></i> 1<br>
     <i style="background:#0096FF;width:15px;height:15px;float:left;margin-right:5px;display:inline-block;"></i> 2<br>
     <i style="background:#0000FF;width:15px;height:15px;float:left;margin-right:5px;display:inline-block;"></i> 3<br>
     <i style="background:#00008B;width:15px;height:15px;float:left;margin-right:5px;display:inline-block;"></i> 4<br>
     <i style="background:#800080;width:15px;height:15px;float:left;margin-right:5px;display:inline-block;"></i> 5 o più<br>
     </div>
    """

    mappa.get_root().html.add_child(folium.Element(legenda_html))

    return mappa

In [ ]:
gdf_risultati = ottimizza_colonnine_con_domanda_aggregata(gdf_mappa = gdf, p_colonnine=1623)

H3 881f99274bfffff: 1 colonnine, 29.5 kWh coperti su 29.5 kWh (Domanda agg: 29.5 kWh)
H3 881f992215fffff: 1 colonnine, 37.4 kWh coperti su 37.4 kWh (Domanda agg: 37.4 kWh)
H3 881f992213fffff: 1 colonnine, 67.3 kWh coperti su 67.3 kWh (Domanda agg: 67.3 kWh)
H3 881f9935c7fffff: 1 colonnine, 18.6 kWh coperti su 18.6 kWh (Domanda agg: 18.6 kWh)
H3 881f9935a1fffff: 1 colonnine, 17.8 kWh coperti su 17.8 kWh (Domanda agg: 17.8 kWh)
H3 881f9922ddfffff: 1 colonnine, 106.8 kWh coperti su 106.8 kWh (Domanda agg: 106.8 kWh)
H3 881f992285fffff: 1 colonnine, 50.0 kWh coperti su 50.0 kWh (Domanda agg: 50.0 kWh)
H3 881f9922e3fffff: 1 colonnine, 18.2 kWh coperti su 18.2 kWh (Domanda agg: 18.2 kWh)
H3 881f992217fffff: 1 colonnine, 20.1 kWh coperti su 20.1 kWh (Domanda agg: 20.1 kWh)
H3 881f993501fffff: 1 colonnine, 71.3 kWh coperti su 71.3 kWh (Domanda agg: 71.3 kWh)
H3 881f99353bfffff: 1 colonnine, 108.0 kWh coperti su 108.0 kWh (Domanda agg: 108.0 kWh)
H3 881f992295fffff: 1 colonnine, 51.4 kWh copert

### Map visualisation

In [ ]:
mappa = crea_mappa_colonnine_discrete(gdf_risultati, colonna_colonnine="Colonnine allocate")
mappa

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# Save the map as an HTML file
mappa.save("/content/drive/Shareddrives/OM in Business Analytics/OM in BA - Project/Codice/04 - Modello/log_heatmap_second_model_def.html")

### Results

In [ ]:
total_allocated_chargers = gdf_risultati["Colonnine allocate"].sum()
total_covered_demand = gdf_risultati["Domanda coperta (kWh)"].sum()
total_demand = gdf_risultati["Domanda Giornaliera Ponderata (kWh)"].sum()

print(f"Total allocated charging stations: {total_allocated_chargers}")
print(f"Total covered demand (kWh): {total_covered_demand:.2f}")
print(f"Total demand (kWh): {total_demand:.2f}")

Total allocated charging stations: 1623
Total covered demand (kWh): 28010.36
Total demand (kWh): 34500.45


## THIRD MODEL  
## Charging Station Allocation Model – Aggregated Neighborhood Demand with Attenuation Factor

### Objective  
Allocate *p* charging stations among the hexagons in the province of Brescia to **maximize the total covered daily electricity demand**, accounting for local demand and **contributions from neighboring hexagons** attenuated by a coefficient *α*.

---

### Decision Variables  
- $ x_h \in \mathbb{Z}_{\geq 0} $: Number of charging stations installed in hexagon *h*  
- $ y_h \in [0, D_h] $: Amount of daily electricity demand covered in hexagon *h*

---

### Parameters  
- $ D_h $: Daily electricity demand in hexagon *h*  
- $ \text{cap} = 555.43 $: Daily capacity of a single charging station (kWh/day)  
- $ p $: Maximum number of charging stations to be installed  
- $ \mathcal{N}(h) $: Set of neighboring hexagons of *h* (within k-ring 1)  
- $ \alpha \in [0, 1] $: Attenuation factor for the contribution of neighboring stations

---

### Constraints  

1. **Total station limit**  
   $$
   \sum_{h=1}^{n} x_h \leq p
   $$

2. **Local demand cannot be exceeded**  
   $$
   y_h \leq D_h \quad \forall h
   $$

3. **Demand covered is limited by capacity of local and neighboring stations**  
   $$
   y_h \leq \text{cap} \cdot \left( x_h + \alpha \cdot \sum_{j \in \mathcal{N}(h)} x_j \right) \quad \forall h
   $$

---

### Notes  
- The model assumes **partial contribution** from neighboring hexagons, modulated by *α*, to reflect diminishing accessibility or influence.  
- The objective promotes **equitable coverage** in areas with concentrated or clustered demand.


### Model function

In [ ]:
import pulp
import pandas as pd

def ottimizza_colonnine_con_alpha(gdf_mappa, neighbours_dict= neighbors_dict, p_colonnine=30, cap=555.43, verbose=True, alpha=0.5 ):
    """
    Risolve il problema di allocazione ottimale delle colonnine considerando la domanda aggregata nel vicinato.

    Parametri:
    - gdf_mappa: GeoDataFrame con 'h3_id' e 'Domanda Giornaliera Ponderata (kWh)'
    - neighbours_dict: dizionario {h3_id: lista di vicini h3_id}
    - p_colonnine: numero totale di colonnine disponibili
    - cap: capacità giornaliera di una colonnina (kWh)
    - verbose: se True, stampa i risultati

    Ritorna:
    - GeoDataFrame con colonne aggiuntive: 'Colonnine allocate', 'Domanda coperta (kWh)', 'Domanda aggregata (kWh)'
    """

    df = gdf_mappa.copy()
    h3_list = df['h3_id'].tolist()
    h3_to_idx = {h3_id: idx for idx, h3_id in enumerate(h3_list)}

    demand = df["Domanda Giornaliera Ponderata (kWh)"].tolist()
    n = len(demand)

    # Calcola domanda aggregata (esagono + vicini)
    domanda_aggregata = []
    for h3_id in h3_list:
        idx = h3_to_idx[h3_id]
        vicini = neighbours_dict.get(h3_id, [])
        domanda_sum = demand[idx]  # domanda locale
        for vicino_id in vicini:
            if vicino_id in h3_to_idx:  # considera solo se vicino presente nel dataset
                domanda_sum += demand[h3_to_idx[vicino_id]]
        domanda_aggregata.append(domanda_sum)
    df["Domanda aggregata (kWh)"] = domanda_aggregata

    # Modello
    model = pulp.LpProblem("Colonnine_con_domanda_aggregata", pulp.LpMaximize)

    # Variabili decisionali
    x = pulp.LpVariable.dicts("Colonnine", range(n), lowBound=0, cat="Integer")  # colonnine allocate
    y = pulp.LpVariable.dicts("DomandaCoperta", range(n), lowBound=0)            # domanda coperta

    # Obiettivo: massimizzare la domanda coperta totale
    model += pulp.lpSum(y[i] for i in range(n))

    # Vincolo: massimo numero di colonnine totali
    model += pulp.lpSum(x[i] for i in range(n)) <= p_colonnine

    # Vincoli per ogni esagono
    for i in range(n):
        h3_id = h3_list[i]
        vicini = neighbours_dict.get(h3_id, [])
        vicini_idx = [h3_to_idx[v] for v in vicini if v in h3_to_idx]

        # Vincolo: massimo 10 colonnine per esagono
        model += x[i] <= 10

        # Domanda coperta deve essere <= domanda locale
        model += y[i] <= demand[i]

        # Domanda coperta deve essere <= capacità totale delle colonnine allocate su esagono + vicini
        model += y[i] <= cap * (x[i] + alpha * pulp.lpSum(x[j] for j in vicini_idx))

    # Risolvi
    model.solve()

    # Estrai risultati
    colonnine_allocate = [int(x[i].varValue) if x[i].varValue else 0 for i in range(n)]
    domanda_coperta = [y[i].varValue if y[i].varValue else 0.0 for i in range(n)]

    df["Colonnine allocate"] = colonnine_allocate
    df["Domanda coperta (kWh)"] = domanda_coperta

    if verbose:
        for i in range(n):
            if colonnine_allocate[i] > 0:
                print(f"H3 {df.iloc[i]['h3_id']}: {colonnine_allocate[i]} colonnine, "
                      f"{domanda_coperta[i]:.1f} kWh coperti su {demand[i]:.1f} kWh (Domanda agg: {domanda_aggregata[i]:.1f} kWh)")
        print(f"\nDomanda totale coperta: {pulp.value(model.objective):.2f} kWh")

    return df

In [ ]:
gdf_risultati_alpha = ottimizza_colonnine_con_alpha(gdf, p_colonnine=1623, alpha=0.167)

H3 881f99274bfffff: 1 colonnine, 29.5 kWh coperti su 29.5 kWh (Domanda agg: 29.5 kWh)
H3 881f992215fffff: 1 colonnine, 37.4 kWh coperti su 37.4 kWh (Domanda agg: 37.4 kWh)
H3 881f992213fffff: 1 colonnine, 67.3 kWh coperti su 67.3 kWh (Domanda agg: 67.3 kWh)
H3 881f9935c7fffff: 1 colonnine, 18.6 kWh coperti su 18.6 kWh (Domanda agg: 18.6 kWh)
H3 881f9935a1fffff: 1 colonnine, 17.8 kWh coperti su 17.8 kWh (Domanda agg: 17.8 kWh)
H3 881f9922ddfffff: 1 colonnine, 106.8 kWh coperti su 106.8 kWh (Domanda agg: 106.8 kWh)
H3 881f992285fffff: 1 colonnine, 50.0 kWh coperti su 50.0 kWh (Domanda agg: 50.0 kWh)
H3 881f9922e3fffff: 1 colonnine, 18.2 kWh coperti su 18.2 kWh (Domanda agg: 18.2 kWh)
H3 881f992217fffff: 1 colonnine, 20.1 kWh coperti su 20.1 kWh (Domanda agg: 20.1 kWh)
H3 881f993501fffff: 1 colonnine, 71.3 kWh coperti su 71.3 kWh (Domanda agg: 71.3 kWh)
H3 881f99353bfffff: 1 colonnine, 108.0 kWh coperti su 108.0 kWh (Domanda agg: 108.0 kWh)
H3 881f992295fffff: 1 colonnine, 51.4 kWh copert

### Map visualisation

In [ ]:
mappa = crea_mappa_colonnine_discrete(gdf_risultati_alpha, colonna_colonnine="Colonnine allocate")
mappa

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# Save the map as an HTML file
mappa.save("/content/drive/Shareddrives/OM in Business Analytics/OM in BA - Project/Codice/04 - Modello/log_heatmap_third_model_def.html")

### Results

In [ ]:
total_allocated_chargers = gdf_risultati_alpha["Colonnine allocate"].sum()
total_covered_demand = gdf_risultati_alpha["Domanda coperta (kWh)"].sum()
total_demand = gdf_risultati_alpha["Domanda Giornaliera Ponderata (kWh)"].sum()

print(f"Total allocated charging stations: {total_allocated_chargers}")
print(f"Total covered demand (kWh): {total_covered_demand:.2f}")
print(f"Total demand (kWh): {total_demand:.2f}")

Total allocated charging stations: 1623
Total covered demand (kWh): 28010.36
Total demand (kWh): 34500.45


# 2030 MODELS

## FOURTH MODEL
## Charging Station Allocation Model – 2030 Conservative Scenario
This model estimates the allocation of 4,039 charging stations under a conservative 2030 scenario. It assumes increased energy demand and enforces a maximum of 10 stations per hexagon. The building index has been excluded, as estimating future building locations and counts would require a separate, dedicated analysis.



In [ ]:
!pip install h3==3.*

### Neighbor list

In [9]:
import h3

# Get the unique list of hexagons in your GeoDataFrame
h3_ids_presenti = set(gdf_conservativo['h3_id'])

# Create a dictionary {index in gdf : list of neighboring indices}
h3_id_to_idx = {h3_id: idx for idx, h3_id in enumerate(gdf_conservativo['h3_id'])}

neighbors_dict = {}

for idx, row in gdf_conservativo.iterrows():
    h3_id = row['h3_id']
    # Get H3 neighbors (k_ring with radius 1)
    vicini_h3 = h3.k_ring(h3_id, 1)
    # Filter only those present in your dataframe
    vicini_presenti = [h3_id_to_idx[v] for v in vicini_h3 if v in h3_ids_presenti and v != h3_id]
    neighbors_dict[idx] = vicini_presenti

print(f"Neighbor dictionary created with {len(neighbors_dict)} hexagons")

Neighbor dictionary created with 6495 hexagons


In [10]:
gdf_risultati_cons2030 = ottimizza_colonnine_con_domanda_aggregata(gdf_conservativo, p_colonnine=4039)

H3 881f99274bfffff: 10 colonnine, 5554.3 kWh coperti su 22330.9 kWh (Domanda agg: 22330.9 kWh)
H3 881f992215fffff: 10 colonnine, 5554.3 kWh coperti su 28225.8 kWh (Domanda agg: 28225.8 kWh)
H3 881f992213fffff: 10 colonnine, 5554.3 kWh coperti su 46257.9 kWh (Domanda agg: 46257.9 kWh)
H3 881f9935c7fffff: 10 colonnine, 5554.3 kWh coperti su 13905.7 kWh (Domanda agg: 13905.7 kWh)
H3 881f9935a1fffff: 10 colonnine, 5554.3 kWh coperti su 13905.7 kWh (Domanda agg: 13905.7 kWh)
H3 881f9922ddfffff: 10 colonnine, 5554.3 kWh coperti su 75974.8 kWh (Domanda agg: 75974.8 kWh)
H3 881f992285fffff: 10 colonnine, 5554.3 kWh coperti su 37835.5 kWh (Domanda agg: 37835.5 kWh)
H3 881f9922e3fffff: 10 colonnine, 5554.3 kWh coperti su 13905.7 kWh (Domanda agg: 13905.7 kWh)
H3 881f992217fffff: 10 colonnine, 5554.3 kWh coperti su 13905.7 kWh (Domanda agg: 13905.7 kWh)
H3 881f993501fffff: 10 colonnine, 5554.3 kWh coperti su 48814.8 kWh (Domanda agg: 48814.8 kWh)
H3 881f99353bfffff: 10 colonnine, 5554.3 kWh coper

### Map visualisation

In [13]:
mappa = crea_mappa_colonnine_discrete(gdf_risultati_cons2030)
mappa

Output hidden; open in https://colab.research.google.com to view.

In [14]:
# Save the map as an HTML file
mappa.save("/content/drive/Shareddrives/OM in Business Analytics/OM in BA - Project/Codice/04 - Modello/log_heatmap_fourth_model_def.html")

### Results

In [ ]:
total_allocated_chargers = gdf_risultati_cons2030["Colonnine allocate"].sum()
total_covered_demand = gdf_risultati_cons2030["Domanda coperta (kWh)"].sum()
total_demand = gdf_risultati_cons2030["Domanda Giornaliera Ponderata (kWh)"].sum()

print(f"Total allocated charging stations: {total_allocated_chargers}")
print(f"Total covered demand (kWh): {total_covered_demand:.2f}")
print(f"Total demand (kWh): {total_demand:.2f}")

Total allocated charging stations: 4039
Total covered demand (kWh): 2243381.77
Total demand (kWh): 10009341.21


## FIFTH MODEL
## Charging Station Allocation Model Alpha – 2030 Conservative Scenario

In [ ]:
gdf_risultati_cons2030_alpha = ottimizza_colonnine_con_alpha(gdf_conservativo, p_colonnine=4039, alpha=0.167)

H3 881f99274bfffff: 10 colonnine, 5554.3 kWh coperti su 22330.9 kWh (Domanda agg: 22330.9 kWh)
H3 881f992215fffff: 10 colonnine, 5554.3 kWh coperti su 28225.8 kWh (Domanda agg: 28225.8 kWh)
H3 881f992213fffff: 10 colonnine, 5554.3 kWh coperti su 46257.9 kWh (Domanda agg: 46257.9 kWh)
H3 881f9935c7fffff: 10 colonnine, 5554.3 kWh coperti su 13905.7 kWh (Domanda agg: 13905.7 kWh)
H3 881f9935a1fffff: 10 colonnine, 5554.3 kWh coperti su 13905.7 kWh (Domanda agg: 13905.7 kWh)
H3 881f9922ddfffff: 10 colonnine, 5554.3 kWh coperti su 75974.8 kWh (Domanda agg: 75974.8 kWh)
H3 881f992285fffff: 10 colonnine, 5554.3 kWh coperti su 37835.5 kWh (Domanda agg: 37835.5 kWh)
H3 881f9922e3fffff: 10 colonnine, 5554.3 kWh coperti su 13905.7 kWh (Domanda agg: 13905.7 kWh)
H3 881f992217fffff: 10 colonnine, 5554.3 kWh coperti su 13905.7 kWh (Domanda agg: 13905.7 kWh)
H3 881f993501fffff: 10 colonnine, 5554.3 kWh coperti su 48814.8 kWh (Domanda agg: 48814.8 kWh)
H3 881f99353bfffff: 10 colonnine, 5554.3 kWh coper

### Map visualisation

In [ ]:
mappa = crea_mappa_colonnine_discrete(gdf_risultati_cons2030_alpha)
mappa

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# Save the map as an HTML file
mappa.save("/content/drive/Shareddrives/OM in Business Analytics/OM in BA - Project/Codice/04 - Modello/log_heatmap_fifth_model_def.html")

### Results

In [ ]:
total_allocated_chargers = gdf_risultati_cons2030_alpha["Colonnine allocate"].sum()
total_covered_demand = gdf_risultati_cons2030_alpha["Domanda coperta (kWh)"].sum()
total_demand = gdf_risultati_cons2030_alpha["Domanda Giornaliera Ponderata (kWh)"].sum()

print(f"Total allocated charging stations: {total_allocated_chargers}")
print(f"Total covered demand (kWh): {total_covered_demand:.2f}")
print(f"Total demand (kWh): {total_demand:.2f}")

Total allocated charging stations: 4039
Total covered demand (kWh): 2243381.77
Total demand (kWh): 10009341.21


## SIXTH MODEL
## Charging Station Allocation Model – 2030 Optimistic Scenario
This model estimates the allocation of 7,345 charging stations under an optimistic 2030 scenario. It assumes increased energy demand. The building index has been excluded, as estimating future building locations and counts would require a separate, dedicated analysis.



In [ ]:
!pip install h3==3.*

### Neighbor list

In [ ]:
import h3

# Get the unique list of hexagons in your GeoDataFrame
h3_ids_presenti = set(gdf_ottimistico['h3_id'])

# Create a dictionary {index in gdf : list of neighboring indices}
h3_id_to_idx = {h3_id: idx for idx, h3_id in enumerate(gdf_ottimistico['h3_id'])}

neighbors_dict = {}

for idx, row in gdf_ottimistico.iterrows():
    h3_id = row['h3_id']
    # Get H3 neighbors (k_ring with radius 1)
    vicini_h3 = h3.k_ring(h3_id, 1)
    # Filter only those present in your dataframe
    vicini_presenti = [h3_id_to_idx[v] for v in vicini_h3 if v in h3_ids_presenti and v != h3_id]
    neighbors_dict[idx] = vicini_presenti

print(f"Neighbor dictionary created with {len(neighbors_dict)} hexagons")

Neighbor dictionary created with 6495 hexagons


In [ ]:
gdf_risultati_ott2030 = ottimizza_colonnine_con_domanda_aggregata(gdf_ottimistico, p_colonnine=7435)

H3 881f99274bfffff: 10 colonnine, 5554.3 kWh coperti su 33496.5 kWh (Domanda agg: 33496.5 kWh)
H3 881f992215fffff: 10 colonnine, 5554.3 kWh coperti su 42338.8 kWh (Domanda agg: 42338.8 kWh)
H3 881f992213fffff: 10 colonnine, 5554.3 kWh coperti su 69387.1 kWh (Domanda agg: 69387.1 kWh)
H3 881f9935c7fffff: 10 colonnine, 5554.3 kWh coperti su 20858.7 kWh (Domanda agg: 20858.7 kWh)
H3 881f9935a1fffff: 10 colonnine, 5554.3 kWh coperti su 20858.7 kWh (Domanda agg: 20858.7 kWh)
H3 881f9922ddfffff: 10 colonnine, 5554.3 kWh coperti su 113962.5 kWh (Domanda agg: 113962.5 kWh)
H3 881f992285fffff: 10 colonnine, 5554.3 kWh coperti su 56753.4 kWh (Domanda agg: 56753.4 kWh)
H3 881f9922e3fffff: 10 colonnine, 5554.3 kWh coperti su 20858.7 kWh (Domanda agg: 20858.7 kWh)
H3 881f992217fffff: 10 colonnine, 5554.3 kWh coperti su 20858.7 kWh (Domanda agg: 20858.7 kWh)
H3 881f993501fffff: 10 colonnine, 5554.3 kWh coperti su 73222.5 kWh (Domanda agg: 73222.5 kWh)
H3 881f99353bfffff: 10 colonnine, 5554.3 kWh cop

### Map visualisation

In [ ]:
mappa = crea_mappa_colonnine_discrete(gdf_risultati_ott2030)
mappa

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# Save the map as an HTML file
mappa.save("/content/drive/Shareddrives/OM in Business Analytics/OM in BA - Project/Codice/04 - Modello/log_heatmap_sixth_model_def.html")

### Results

In [ ]:
total_allocated_chargers = gdf_risultati_ott2030["Colonnine allocate"].sum()
total_covered_demand = gdf_risultati_ott2030["Domanda coperta (kWh)"].sum()
total_demand = gdf_risultati_ott2030["Domanda Giornaliera Ponderata (kWh)"].sum()

print(f"Total allocated charging stations: {total_allocated_chargers}")
print(f"Total covered demand (kWh): {total_covered_demand:.2f}")
print(f"Total demand (kWh): {total_demand:.2f}")

Total allocated charging stations: 7435
Total covered demand (kWh): 4129622.05
Total demand (kWh): 15014065.22


## SEVENTH MODEL
## Charging Station Allocation Model Alpha – 2030 Optimistic Scenario

In [ ]:
gdf_risultati_ott2030_alpha = ottimizza_colonnine_con_alpha(gdf_ottimistico, p_colonnine=7435, alpha=0.167)

H3 881f99274bfffff: 10 colonnine, 5554.3 kWh coperti su 33496.5 kWh (Domanda agg: 33496.5 kWh)
H3 881f992215fffff: 10 colonnine, 5554.3 kWh coperti su 42338.8 kWh (Domanda agg: 42338.8 kWh)
H3 881f992213fffff: 10 colonnine, 5554.3 kWh coperti su 69387.1 kWh (Domanda agg: 69387.1 kWh)
H3 881f9935c7fffff: 10 colonnine, 5554.3 kWh coperti su 20858.7 kWh (Domanda agg: 20858.7 kWh)
H3 881f9935a1fffff: 10 colonnine, 5554.3 kWh coperti su 20858.7 kWh (Domanda agg: 20858.7 kWh)
H3 881f9922ddfffff: 10 colonnine, 5554.3 kWh coperti su 113962.5 kWh (Domanda agg: 113962.5 kWh)
H3 881f992285fffff: 10 colonnine, 5554.3 kWh coperti su 56753.4 kWh (Domanda agg: 56753.4 kWh)
H3 881f9922e3fffff: 10 colonnine, 5554.3 kWh coperti su 20858.7 kWh (Domanda agg: 20858.7 kWh)
H3 881f992217fffff: 10 colonnine, 5554.3 kWh coperti su 20858.7 kWh (Domanda agg: 20858.7 kWh)
H3 881f993501fffff: 10 colonnine, 5554.3 kWh coperti su 73222.5 kWh (Domanda agg: 73222.5 kWh)
H3 881f99353bfffff: 10 colonnine, 5554.3 kWh cop

### Map visualisation

In [ ]:
mappa = crea_mappa_colonnine_discrete(gdf_risultati_ott2030_alpha)
mappa

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# Save the map as an HTML file
mappa.save("/content/drive/Shareddrives/OM in Business Analytics/OM in BA - Project/Codice/04 - Modello/log_heatmap_seventh_model_def.html")

### Results

In [ ]:
total_allocated_chargers = gdf_risultati_ott2030_alpha["Colonnine allocate"].sum()
total_covered_demand = gdf_risultati_ott2030_alpha["Domanda coperta (kWh)"].sum()
total_demand = gdf_risultati_ott2030_alpha["Domanda Giornaliera Ponderata (kWh)"].sum()

print(f"Total allocated charging stations: {total_allocated_chargers}")
print(f"Total covered demand (kWh): {total_covered_demand:.2f}")
print(f"Total demand (kWh): {total_demand:.2f}")

Total allocated charging stations: 7435
Total covered demand (kWh): 4129622.05
Total demand (kWh): 15014065.22
